# NORMALIZE AND CLEAN DATA

In [1]:
import pandas as pd
import sys
import os

# Add project root to Python path to allow module imports
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import custom utility functions for data processing
from src.utils.normalizing import normalize          # Standardize or scale data
from src.utils.qa_rules import run_quality_check, summarize_qa_flags  # Apply and summarize QA rules
from src.utils.cleaning import clean                 # Perform data cleaning

## Normalize Data of a month using built functions
**Objective:** Load the raw January data and apply the `normalize` function (from `src.utils.normalizing`).
* **Enrich data:** Add `PU/DO_Borough`, `payment_type_name`, etc.
* **Feature Engineering:** Create derived columns like `trip_duration`, `avg_speed`, and `pickup_day_of_week`.
* **Feature Selection:** Drop irrelevant columns identified in Notebook 1, specifically 'airport_fee' since it has only 5 nonnull values; 'store_and_fwd_flag', 'VendorID' because they are irrelevant to analysis; 'mta_tax','improvement_surcharge' due to their low variance.

In [2]:
df1 = pd.read_parquet("../raw/yellow_tripdata_2021-01.parquet")
df1_normalized = normalize(df1)
print("Successfully normalized January data")

Successfully normalized January data


In [3]:
print("First 10 rows of data before normalized: ")
df1.head(10)

First 10 rows of data before normalized: 


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5,NaN
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5,NaN
5,1,2021-01-01 00:16:29,2021-01-01 00:24:30,1.0,1.60,1.0,N,224,68,1,8.0,3.0,0.5,2.35,0.0,0.3,14.15,2.5,NaN
6,1,2021-01-01 00:00:28,2021-01-01 00:17:28,1.0,4.10,1.0,N,95,157,2,16.0,0.5,0.5,0.00,0.0,0.3,17.30,0.0,NaN
7,1,2021-01-01 00:12:29,2021-01-01 00:30:34,1.0,5.70,1.0,N,90,40,2,18.0,3.0,0.5,0.00,0.0,0.3,21.80,2.5,NaN
8,1,2021-01-01 00:39:16,2021-01-01 01:00:13,1.0,9.10,1.0,N,97,129,4,27.5,0.5,0.5,0.00,0.0,0.3,28.80,0.0,NaN
9,1,2021-01-01 00:26:12,2021-01-01 00:39:46,2.0,2.70,1.0,N,263,142,1,12.0,3.0,0.5,3.15,0.0,0.3,18.95,2.5,NaN


In [4]:
print("First 10 rows of data after normalized with new columns at the end: ")
df1_normalized.head(10)

First 10 rows of data after normalized with new columns at the end: 


,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,ratecodeID_name,payment_type_name,PU_Borough,PU_Zone,DO_Borough,DO_Zone,trip_duration_seconds,trip_duration_minutes,avg_speed_mph,pickup_day_of_week
0,2021-01-01 00:30:10,2021-01-01 00:36:12,1.0,2.10,1.0,142,43,2,8.0,3.0,...,Standard rate,Cash,Manhattan,Lincoln Square East,Manhattan,Central Park,362.0,6.0,20.88,Friday
1,2021-01-01 00:51:20,2021-01-01 00:52:19,1.0,0.20,1.0,238,151,2,3.0,0.5,...,Standard rate,Cash,Manhattan,Upper West Side North,Manhattan,Manhattan Valley,59.0,1.0,12.20,Friday
2,2021-01-01 00:43:30,2021-01-01 01:11:06,1.0,14.70,1.0,132,165,1,42.0,0.5,...,Standard rate,Credit card,Queens,JFK Airport,Brooklyn,Midwood,1656.0,28.0,31.96,Friday
3,2021-01-01 00:15:48,2021-01-01 00:31:01,0.0,10.60,1.0,138,132,1,29.0,0.5,...,Standard rate,Credit card,Queens,LaGuardia Airport,Queens,JFK Airport,913.0,15.0,41.80,Friday
4,2021-01-01 00:31:49,2021-01-01 00:48:21,1.0,4.94,1.0,68,33,1,16.5,0.5,...,Standard rate,Credit card,Manhattan,East Chelsea,Brooklyn,Brooklyn Heights,992.0,17.0,17.93,Friday
5,2021-01-01 00:16:29,2021-01-01 00:24:30,1.0,1.60,1.0,224,68,1,8.0,3.0,...,Standard rate,Credit card,Manhattan,Stuy Town/Peter Cooper Village,Manhattan,East Chelsea,481.0,8.0,11.98,Friday
6,2021-01-01 00:00:28,2021-01-01 00:17:28,1.0,4.10,1.0,95,157,2,16.0,0.5,...,Standard rate,Cash,Queens,Forest Hills,Queens,Maspeth,1020.0,17.0,14.47,Friday
7,2021-01-01 00:12:29,2021-01-01 00:30:34,1.0,5.70,1.0,90,40,2,18.0,3.0,...,Standard rate,Cash,Manhattan,Flatiron,Brooklyn,Carroll Gardens,1085.0,18.0,18.91,Friday
8,2021-01-01 00:39:16,2021-01-01 01:00:13,1.0,9.10,1.0,97,129,4,27.5,0.5,...,Standard rate,Dispute,Brooklyn,Fort Greene,Queens,Jackson Heights,1257.0,21.0,26.06,Friday
9,2021-01-01 00:26:12,2021-01-01 00:39:46,2.0,2.70,1.0,263,142,1,12.0,3.0,...,Standard rate,Credit card,Manhattan,Yorkville West,Manhattan,Lincoln Square East,814.0,14.0,11.94,Friday


In [5]:
print("Data after normalized info: ")
df1_normalized.info()

Data after normalized info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1369769 entries, 0 to 1369768
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   tpep_pickup_datetime   1369769 non-null  datetime64[ns]
 1   tpep_dropoff_datetime  1369769 non-null  datetime64[ns]
 2   passenger_count        1271417 non-null  float64       
 3   trip_distance          1369769 non-null  float64       
 4   RatecodeID             1271417 non-null  float64       
 5   PULocationID           1369769 non-null  int64         
 6   DOLocationID           1369769 non-null  int64         
 7   payment_type           1369769 non-null  int64         
 8   fare_amount            1369769 non-null  float64       
 9   extra                  1369769 non-null  float64       
 10  tip_amount             1369769 non-null  float64       
 11  tolls_amount           1369769 non-null  float64       
 12 

## Applying QA steps 
**Objective:** Apply the `run_quality_check` function (from `src.utils.qa_rules`) to the normalized data.
* This will return `df1_flag`, a DataFrame containing 11 boolean flag columns based on 11 rules.
* Then, use `summarize_qa_flags` to generate the summary string ("count/pct%") for the report.

In [6]:
df1_flag = run_quality_check(df1_normalized)
january = summarize_qa_flags(df1_flag)
print("Successfully run quality check!")

Successfully run quality check!


In [7]:
print("First 10 rows of January's flag: ")
df1_flag.head(10)

First 10 rows of January's flag: 


,is_duplicate,invalid_time_order,invalid_duration,invalid_distance,invalid_speed,invalid_fare_amount,invalid_tip_amount,invalid_total_amount,invalid_payment_type,invalid_passenger_count,invalid_zone
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False


In [8]:
print("Summary of January's flag, 0-10 indicates rule ID, row 11 is the sum of trips that has violations: ")
january

Summary of January's flag, 0-10 indicates rule ID, row 11 is the sum of trips that has violations: 


0          0/0.0%
1      5642/0.41%
2     16119/1.18%
3     19952/1.46%
4     24893/1.82%
5      7411/0.54%
6         59/0.0%
7      7114/0.52%
8          0/0.0%
9     26726/1.95%
10    27137/1.98%
11    79108/5.78%
dtype: object

## Clean Data of a month using built function
**Objective:** Apply the `clean` function (from `src.utils.cleaning`).
* This function will take `df1_normalized` and `df1_flag` as input.
* It will filter and remove rows that violate the rules according to our defined strategy.

In [9]:
df1_cleaned, df1_standard = clean(df1_normalized, df1_flag)
print("Successfully cleaned data!")
print("Cleaned data has shape: ", df1_cleaned.shape)

Successfully cleaned data!
Cleaned data has shape:  (1340386, 24)


In [10]:
print("Data after cleaned info: ")
df1_cleaned.info()

Data after cleaned info: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1340386 entries, 0 to 1369768
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   tpep_pickup_datetime   1340386 non-null  datetime64[ns]
 1   tpep_dropoff_datetime  1340386 non-null  datetime64[ns]
 2   passenger_count        1249613 non-null  float64       
 3   trip_distance          1340386 non-null  float64       
 4   RatecodeID             1249613 non-null  float64       
 5   PULocationID           1340386 non-null  int64         
 6   DOLocationID           1340386 non-null  int64         
 7   payment_type           1340386 non-null  int64         
 8   fare_amount            1340386 non-null  float64       
 9   extra                  1340386 non-null  float64       
 10  tip_amount             1340386 non-null  float64       
 11  tolls_amount           1340386 non-null  float64       
 12  to

In [11]:
print("More information about cleaned data (numerical values):")
df1_cleaned.describe()

More information about cleaned data (numerical values):


,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,tip_amount,tolls_amount,total_amount,congestion_surcharge,trip_duration_seconds,trip_duration_minutes,avg_speed_mph
count,1.249613e+06,1.340386e+06,1.249613e+06,1.340386e+06,1.340386e+06,1.340386e+06,1.340386e+06,1.340386e+06,1.340386e+06,1.340386e+06,1.340386e+06,1.249613e+06,1.340386e+06,1.340386e+06,1.340386e+06
mean,1.415902e+00,4.704354e+00,1.022069e+00,1.650348e+02,1.617922e+02,1.189912e+00,1.192878e+01,9.854489e-01,1.940023e+00,2.433507e-01,1.736598e+01,2.269395e+00,8.455518e+02,1.409220e+01,1.911607e+01
std,1.064091e+00,3.981971e+02,3.911283e-01,6.751307e+01,7.193394e+01,5.668484e-01,1.250073e+01,1.233965e+00,2.533959e+00,1.514854e+00,1.430860e+01,7.562342e-01,3.759726e+03,6.266288e+01,1.417377e+03
min,0.000000e+00,1.000000e-02,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.500000e+02,-5.500000e+00,-3.072000e+01,-3.112000e+01,-1.764200e+02,-2.500000e+00,3.100000e+01,1.000000e+00,1.000000e-02
25%,1.000000e+00,1.020000e+00,1.000000e+00,1.250000e+02,1.070000e+02,1.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.080000e+01,2.500000e+00,3.410000e+02,6.000000e+00,9.590000e+00
50%,1.000000e+00,1.700000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,8.500000e+00,5.000000e-01,1.860000e+00,0.000000e+00,1.380000e+01,2.500000e+00,5.480000e+02,9.000000e+00,1.189000e+01
75%,1.000000e+00,3.050000e+00,1.000000e+00,2.360000e+02,2.360000e+02,1.000000e+00,1.300000e+01,2.500000e+00,2.750000e+00,0.000000e+00,1.896000e+01,2.500000e+00,8.770000e+02,1.500000e+01,1.530000e+01
max,8.000000e+00,2.631633e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,6.960500e+03,8.250000e+00,1.140440e+03,8.117500e+02,7.661280e+03,2.500000e+00,1.729062e+06,2.881800e+04,1.054122e+06


## Clean Data of 12 months and save it to processed


In [21]:
from pathlib import Path

raw_dir = Path("../raw")
raw_files = list(raw_dir.glob("yellow_tripdata_2021-*.parquet"))

cleaned_dir = Path("../processed/cleaned_data")
flag_dir = Path("../processed/flags_for_analysis")

reports_dir = Path("../reports")

In [ ]:
id = [1,2,3,4,5,6,7,8,9,10,11,'NaN']
rule = ['is_duplicate', 'invalid_time_order', 'invalid_duration', 'invalid_distance', 
    'invalid_speed', 'invalid_fare_amount', 'invalid_tip_amount', 'invalid_total_amount', 
    'invalid_payment_type', 'invalid_passenger_count', 'invalid_zone (Unknown)', 'NaN']

action = ['Exclude', 'Exclude', 'Exclude', 'Exclude', 
        'Exclude', 'Flag', 'Flag', 'Flag', 'Flag',
        'Flag', 'Flag', 'Total:']

rule_definition = {'ID': id, 'Rule': rule, 'Action': action}
final_qa_report_df = pd.DataFrame(data = rule_definition, columns=['ID', 'Rule', 'Action'])
final_qa_report_df


,ID,Rule,Action
0,1,is_duplicate,Exclude
1,2,invalid_time_order,Exclude
2,3,invalid_duration,Exclude
3,4,invalid_distance,Exclude
4,5,invalid_speed,Exlucde
5,6,invalid_fare_amount,Flag
6,7,invalid_tip_amount,Flag
7,8,invalid_total_amount,Flag
8,9,invalid_payment_type,Flag
9,10,invalid_passenger_count,Flag


In [ ]:
for file in raw_files:
    # Take month's name:
    month_str = file.name.split('-')[1].split('.')[0]
    month_col_name = pd.to_datetime(f'2021-{month_str}-01').strftime('%B') 
    month_int = pd.to_datetime(f"2021-{month_str}-01").month

    print(f"--- Processing: {month_col_name}: {file.name} ---")
    df_file = pd.read_parquet(file)
    
    # Normalize data:
    normalized_file = normalize(df_file)

    # Applying QA rules, add QA summary to final QA report:
    file_flag = run_quality_check(normalized_file, month_int)
    flag_summary = summarize_qa_flags(file_flag)
    final_qa_report_df[month_col_name] = flag_summary

    # Clean data and take flag of not removed rows:
    file_cleaned, file_standard = clean(normalized_file, file_flag)

    # Save cleaned file to folder processed/cleaned_data
    cleaned_out = cleaned_dir / f"cleaned_{file.name}"
    if cleaned_out.exists():
        print(f"Skipped (already exists): {cleaned_out.name}")
    else:
        file_cleaned.to_parquet(cleaned_out, index=False, engine="pyarrow")
        print(f"Saved cleaned file: {cleaned_out.name}")
    
    # Save flags for later analysis to folder processed/flags_for_analysis
    flag_out = flag_dir / f"flag_{file.name}"
    if flag_out.exists():
        print(f"Skipped (already exists): {flag_out.name}")
    else:
        file_standard.to_parquet(flag_out, index=False, engine="pyarrow")
        print(f"Saved flag file for upcoming analysis: {flag_out.name}")

# Save QA summary to folder reports
qa_path = reports_dir / "qa_summary.csv"
final_qa_report_df.to_csv(qa_path, index=False)
print(f"QA report saved to {qa_path}")
print("Data is ready for analysis!")


--- Processing: January: yellow_tripdata_2021-01.parquet ---
Saved cleaned file: cleaned_yellow_tripdata_2021-01.parquet
Saved flag file for upcoming analysis: flag_yellow_tripdata_2021-01.parquet
--- Processing: February: yellow_tripdata_2021-02.parquet ---
Saved cleaned file: cleaned_yellow_tripdata_2021-02.parquet
Saved flag file for upcoming analysis: flag_yellow_tripdata_2021-02.parquet
--- Processing: March: yellow_tripdata_2021-03.parquet ---
Saved cleaned file: cleaned_yellow_tripdata_2021-03.parquet
Saved flag file for upcoming analysis: flag_yellow_tripdata_2021-03.parquet
--- Processing: April: yellow_tripdata_2021-04.parquet ---
Saved cleaned file: cleaned_yellow_tripdata_2021-04.parquet
Saved flag file for upcoming analysis: flag_yellow_tripdata_2021-04.parquet
--- Processing: May: yellow_tripdata_2021-05.parquet ---
Saved cleaned file: cleaned_yellow_tripdata_2021-05.parquet
Saved flag file for upcoming analysis: flag_yellow_tripdata_2021-05.parquet
--- Processing: June: 

In [32]:
df3 = pd.read_parquet("../processed/cleaned_data/cleaned_yellow_tripdata_2021-05.parquet")
df3.head(10)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,ratecodeID_name,payment_type_name,PU_Borough,PU_Zone,DO_Borough,DO_Zone,trip_duration_seconds,trip_duration_minutes,avg_speed_mph,pickup_day_of_week
0,2021-05-01 00:37:18,2021-05-01 00:41:07,2.0,0.70,1.0,141,263,1,5.0,3.0,...,Standard rate,Credit card,Manhattan,Lenox Hill West,Manhattan,Yorkville West,229.0,4.0,11.00,Saturday
1,2021-05-01 00:43:01,2021-05-01 00:49:19,1.0,1.40,1.0,263,75,2,6.5,3.0,...,Standard rate,Cash,Manhattan,Yorkville West,Manhattan,East Harlem South,378.0,6.0,13.33,Saturday
2,2021-05-01 00:05:54,2021-05-01 00:31:46,1.0,5.70,1.0,142,129,2,21.5,3.0,...,Standard rate,Cash,Manhattan,Lincoln Square East,Queens,Jackson Heights,1552.0,26.0,13.22,Saturday
3,2021-05-01 00:08:21,2021-05-01 00:19:20,1.0,3.04,1.0,231,97,1,11.5,0.5,...,Standard rate,Credit card,Manhattan,TriBeCa/Civic Center,Brooklyn,Fort Greene,659.0,11.0,16.61,Saturday
4,2021-05-01 00:32:44,2021-05-01 00:48:44,1.0,4.04,1.0,148,17,1,15.5,0.5,...,Standard rate,Credit card,Manhattan,Lower East Side,Brooklyn,Bedford,960.0,16.0,15.15,Saturday
5,2021-05-01 00:08:33,2021-05-01 00:19:25,0.0,3.10,1.0,231,68,1,11.5,3.0,...,Standard rate,Credit card,Manhattan,TriBeCa/Civic Center,Manhattan,East Chelsea,652.0,11.0,17.12,Saturday
6,2021-05-01 00:34:16,2021-05-01 00:42:12,0.0,2.10,1.0,142,68,1,8.5,3.0,...,Standard rate,Credit card,Manhattan,Lincoln Square East,Manhattan,East Chelsea,476.0,8.0,15.88,Saturday
7,2021-05-01 00:25:53,2021-05-01 00:36:21,1.0,3.60,1.0,148,112,1,13.0,3.0,...,Standard rate,Credit card,Manhattan,Lower East Side,Brooklyn,Greenpoint,628.0,10.0,20.64,Saturday
8,2021-05-01 00:49:19,2021-05-01 01:01:18,2.0,3.10,1.0,148,229,1,12.0,3.0,...,Standard rate,Credit card,Manhattan,Lower East Side,Manhattan,Sutton Place/Turtle Bay North,719.0,12.0,15.52,Saturday
9,2021-05-01 00:06:23,2021-05-01 00:18:50,1.0,2.90,1.0,68,143,2,11.5,3.0,...,Standard rate,Cash,Manhattan,East Chelsea,Manhattan,Lincoln Square West,747.0,12.0,13.98,Saturday


In [36]:
df3.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'tip_amount', 'tolls_amount',
       'total_amount', 'congestion_surcharge', 'ratecodeID_name',
       'payment_type_name', 'PU_Borough', 'PU_Zone', 'DO_Borough', 'DO_Zone',
       'trip_duration_seconds', 'trip_duration_minutes', 'avg_speed_mph',
       'pickup_day_of_week'],
      dtype='object')